In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: joshuaml
Azure region: eastus2
Subscription id: c9406ee3-cb71-408a-a29d-74c8b550c320
Resource group: joshuaml


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "JoshuaML"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
#compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2022-01-17T08:38:20.476000+00:00', 'errors': None, 'creationTime': '2022-01-17T08:38:19.076911+00:00', 'modifiedTime': '2022-01-17T08:38:23.604031+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:
from azureml.core import Environment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import ScriptRunConfig
import os
import shutil

project_folder = "./training"
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', "./training")

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.50,2),
    "--max_iter": choice(range(100,400))
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_amount=0.10, delay_evaluation=0)

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [5]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [6]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_841e1fab-c4a4-4143-9364-e0a3e1044126
Web View: https://ml.azure.com/runs/HD_841e1fab-c4a4-4143-9364-e0a3e1044126?wsid=/subscriptions/c9406ee3-cb71-408a-a29d-74c8b550c320/resourcegroups/joshuaml/workspaces/joshuaml&tid=5167a66f-432b-4c20-85ac-05c76c77a81d

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-17T08:38:58.438651][API][INFO]Experiment created<END>\n""<START>[2022-01-17T08:38:59.113275][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-17T08:38:59.677330][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_841e1fab-c4a4-4143-9364-e0a3e1044126
Web View: https://ml.azure.com/runs/HD_841e1fab-c4a4-4143-9364-e0a3e1044126?wsid=/subscriptions/c9406ee3-cb71-408a-a29d-74c8b550c320/resourcegroups/joshuaml/workspaces/joshuaml&tid=5167a66f-432b-4c20-85ac-05c76c77a81d



{'runId': 'HD_841e1fab-c4a4-4143-9364-e0a3e1044126',
 'target': 'JoshuaML',
 'status': 'Completed',
 'startTimeUtc': '2022-01-17T08:38:58.258318Z',
 'endTimeUtc': '2022-01-17T08:53:00.456207Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '59242dba-78a8-469a-96ab-a0c5430a794b',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': 'infinite_space_size',
  'score': '0.9099295945617868',
  'best_child_run_id': 'HD_841e1fab-c4a4-4143-9364-e0a3e1044126_2',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://joshuaml6462248134.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_841e1fab-c4a4-4143-9364-e0

In [7]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '1', '--max_iter', '100', '--C', '1.4405660351264873', '--max_iter', '265']


In [8]:
best_run.get_file_names()

# why does model not save?S

['logs/azureml/20_azureml.log',
 'logs/azureml/dataprep/backgroundProcess.log',
 'logs/azureml/dataprep/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/rslex.log',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'user_logs/std_log.txt']

In [9]:
import joblib
# Get your best run and save the model from that run.

model = best_run.register_model(model_name='hyperdrive_model', model_path='./outputs/model.joblib', tags=best_run.get_metrics())

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
import pandas as pd

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

x, y = clean_data(ds)
data = pd.concat([x, y], axis=1)

os.makedirs('./processed_data/', exist_ok=True)
data.to_csv('./processed_data/bankmarketing_train.csv')

In [11]:
from azureml.core import Dataset

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['processed_data/bankmarketing_train.csv'],
                       target_path = 'bankmarketing_train/tabular/',
                       overwrite = True,
                       show_progress = True)

dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'bankmarketing_train/tabular/bankmarketing_train.csv')])

Uploading an estimated of 1 files
Uploading processed_data/bankmarketing_train.csv
Uploaded processed_data/bankmarketing_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=3)

In [13]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_61d94bd3-1410-4a6e-8845-7c7f76d48104,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
remote_run.wait_for_completion()

{'runId': 'AutoML_61d94bd3-1410-4a6e-8845-7c7f76d48104',
 'target': 'JoshuaML',
 'status': 'Completed',
 'startTimeUtc': '2022-01-17T08:54:15.746682Z',
 'endTimeUtc': '2022-01-17T09:21:07.144738Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'JoshuaML',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"c9406ee3-cb71-408a-a29d-74c8b550c320","resource_group":"joshuaml","workspace_name":"joshuaml","region":"eastus2","compute_target":"JoshuaML","spark_service":null,"azure_service":"remote"

In [15]:
# Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.37.0, current version:1.36.0
Package:azureml-core, training version:1.37.0, current version:1.36.0.post2
Package:azureml-dataprep, training version:2.25.0, current version:2.24.4
Package:azureml-dataprep-rslex, training version:2.1.0, current version:2.0.3
Package:azureml-dataset-runtime, training version:1.37.0, current version:1.36.0
Package:azureml-defaults, training version:1.37.0, current version:1.36.0
Package:azureml-interpret, training version:1.37.0, current version:1.36.0
Package:azureml-mlflow, training version:1.37.0, current version:1.36.0
Package:azureml-pipeline-core, training version:1.37.0, current version:1.36.0
Package:azureml-responsibleai, training version:1.37.0, current version:1.36.0
Package:azureml-telemetry, training version:1.37.0, current version:1.36.0
Package:azureml-train-automl-client, training version:1.37.0, current version:1.36.0
Package:azureml-train-automl-runtime, training version:1.37.0, current v

In [16]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=2.1875, reg_lambda=1.0416666666666667, subsample=1, tree_method='auto'))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.5510448979591835, class_weight='balanced', eta0=0.001, fit_intercept=True, l1_ratio=0.42857142857142855, learning_rate='constant', loss='modified_huber', max_iter=1000, n_jobs=1, penalty='none', power_t=0.7777777777777777, random_state=None, tol=0.0001))], verbose=False))], flatten_transform=None, weights=[0.2, 0.3333333333333333, 0.06666666666666667, 

In [17]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_61d94bd3-1410-4a6e-8845-7c7f76d48104_30,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
compute_target.delete()